In [1]:
import numpy as np
from typing import List, Tuple, Dict

## Play Tennis Sample

In [2]:
def create_training_data():
    """Create the training dataset for tennis prediction."""
    data = [
        ['Sunny', 'Hot', 'High', 'Weak', 'No'],
        ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        ['Overcast', 'Mild', 'High', 'Weak', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'Normal', 'Weak', 'Yes']
    ]
    return np.array(data)

train_data = create_training_data()
print(train_data)

[['Sunny' 'Hot' 'High' 'Weak' 'No']
 ['Sunny' 'Hot' 'High' 'Strong' 'No']
 ['Overcast' 'Hot' 'High' 'Weak' 'Yes']
 ['Rain' 'Mild' 'High' 'Weak' 'Yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'No']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'Yes']
 ['Overcast' 'Mild' 'High' 'Weak' 'No']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'Yes']]


In [3]:
def compute_prior_probabilities(train_data):
    """
    Calculate prior probabilities P(Play Tennis = Yes/No).

    Args:
        train_data: Training dataset

    Returns:
        Array of prior probabilities [P(No), P(Yes)]
    """

    ### Your code here
    class_names = ['No', 'Yes']
    total_samples = len(train_data)
    prior_probs = np.zeros(len(class_names))
    for i, class_name in enumerate(class_names):
        class_count = np.sum(train_data[:, -1] == class_name)
        prior_probs[i] = class_count / total_samples

    return prior_probs

prior_probablity = compute_prior_probabilities(train_data)
print("P(“Play Tennis” = Yes)", prior_probablity[1])
print("P(“Play Tennis” = No)", prior_probablity[0])

P(“Play Tennis” = Yes) 0.6
P(“Play Tennis” = No) 0.4


In [4]:
prior_probablity

array([0.4, 0.6])

In [5]:
def compute_conditional_probabilities(train_data):
    """
    Calculate conditional probabilities P(Feature|Class) for all features.

    Args:
        train_data: Training dataset

    Returns:
        Tuple of (conditional_probabilities, feature_values)
    """
    class_names = ['No', 'Yes']
    n_features = train_data.shape[1] - 1  # Exclude target column
    conditional_probs = []
    feature_values = []

    for feature_idx in range(n_features): # 0, 1, 2, 3
        # Get unique values for this feature
        unique_values = np.unique(train_data[:, feature_idx]) # 0: S, O R
        feature_values.append(unique_values)

        # Initialize conditional probability matrix
        feature_cond_probs = np.zeros((len(class_names), len(unique_values)))
        # 0: 2x3

        for class_idx, class_name in enumerate(class_names):
            # Get samples for this class
            class_mask = train_data[:, -1] == class_name
            class_samples = train_data[class_mask]

            # Count occurrences of each feature value in this class

            for value_idx, value in enumerate(unique_values):
                # Count occurrences of this feature value in this class
                # 0: S, O, R
                feature_count = np.sum(class_samples[:, feature_idx] == value)
                # Calculate conditional probability
                feature_cond_probs[class_idx, value_idx] = feature_count / len(class_samples)

        conditional_probs.append(feature_cond_probs)

    return conditional_probs, feature_values

In [6]:
_, feature_values  = compute_conditional_probabilities(train_data)
for x in _:
    print(x)

for feature in feature_values:
    print(feature)

[[0.25       0.25       0.5       ]
 [0.33333333 0.5        0.16666667]]
[[0.25       0.5        0.25      ]
 [0.5        0.16666667 0.33333333]]
[[0.75       0.25      ]
 [0.33333333 0.66666667]]
[[0.5        0.5       ]
 [0.16666667 0.83333333]]
['Overcast' 'Rain' 'Sunny']
['Cool' 'Hot' 'Mild']
['High' 'Normal']
['Strong' 'Weak']


In [7]:
def train_naive_bayes(train_data):
    """
    Train the Naive Bayes classifier.

    Args:
        train_data: Training dataset

    Returns:
        Tuple of (prior_probabilities, conditional_probabilities, feature_names)
    """

    # Calculate prior probabilities
    prior_probabilities = compute_prior_probabilities(train_data)

    # Calculate conditional probabilities
    conditional_probabilities, feature_names = compute_conditional_probabilities(train_data)

    return prior_probabilities, conditional_probabilities, feature_names

In [8]:
# Train the model
prior_probs, conditional_probs, feature_names = train_naive_bayes(train_data)

print(f"prior_probablity: {prior_probablity}")
print(f"conditional_probs: {conditional_probs}")
print(f"feature_names: {feature_names}")

prior_probablity: [0.4 0.6]
conditional_probs: [array([[0.25      , 0.25      , 0.5       ],
       [0.33333333, 0.5       , 0.16666667]]), array([[0.25      , 0.5       , 0.25      ],
       [0.5       , 0.16666667, 0.33333333]]), array([[0.75      , 0.25      ],
       [0.33333333, 0.66666667]]), array([[0.5       , 0.5       ],
       [0.16666667, 0.83333333]])]
feature_names: [array(['Overcast', 'Rain', 'Sunny'], dtype='<U8'), array(['Cool', 'Hot', 'Mild'], dtype='<U8'), array(['High', 'Normal'], dtype='<U8'), array(['Strong', 'Weak'], dtype='<U8')]


In [9]:
def get_feature_index(feature_value, feature_values):
    """
    Get the index of a feature value in the feature values array.

    Args:
        feature_value: Value to find
        feature_values: Array of possible feature values

    Returns:
        Index of the feature value
    """
    return np.where(feature_values == feature_value)[0][0]

In [10]:
feature_values

[array(['Overcast', 'Rain', 'Sunny'], dtype='<U8'),
 array(['Cool', 'Hot', 'Mild'], dtype='<U8'),
 array(['High', 'Normal'], dtype='<U8'),
 array(['Strong', 'Weak'], dtype='<U8')]

In [11]:
outlook = feature_values[0]
i1 = get_feature_index("Overcast", outlook)
i2 = get_feature_index("Rain", outlook)
i3 = get_feature_index("Sunny", outlook)

print(i1, i2, i3)

0 1 2


In [12]:
# Compute P("Outlook"="Sunny"|Play Tennis"="Yes")
x1 = get_feature_index("Sunny",feature_values[0])
print("P('Outlook'='Sunny'|Play Tennis'='Yes') = ",
      np.round(conditional_probs[0][1, x1],2)
)

P('Outlook'='Sunny'|Play Tennis'='Yes') =  0.17


In [13]:
conditional_probs

[array([[0.25      , 0.25      , 0.5       ],
        [0.33333333, 0.5       , 0.16666667]]),
 array([[0.25      , 0.5       , 0.25      ],
        [0.5       , 0.16666667, 0.33333333]]),
 array([[0.75      , 0.25      ],
        [0.33333333, 0.66666667]]),
 array([[0.5       , 0.5       ],
        [0.16666667, 0.83333333]])]

In [14]:
# Compute P("Outlook"="Sunny"|Play Tennis"="No")
x1 = get_feature_index("Sunny",feature_values[0])
print(
    "P('Outlook'='Sunny'|Play Tennis'='No') = ",
    np.round(conditional_probs[0][0, x1],2)
)

P('Outlook'='Sunny'|Play Tennis'='No') =  0.5


In [15]:
def predict_tennis(
        X, prior_probabilities, conditional_probabilities, feature_names
    ):
    """
    Make a prediction for given features.

    Args:
        X: List of feature values [Outlook, Temperature, Humidity, Wind]
        prior_probabilities: Prior probabilities for each class
        conditional_probabilities: Conditional probabilities for each feature
        feature_names: Names/values for each feature

    Returns:
        Tuple of (prediction, probabilities)
    """
    class_names = ['No', 'Yes']

    # Get feature indices
    feature_indices = [] # 2, 1, 1, 1
    for i, feature_value in enumerate(X):
        feature_indices.append(get_feature_index(feature_value, feature_names[i]))

    # Calculate probabilities for each class
    class_probabilities = []

    for class_idx in range(len(class_names)):
        # Start with prior probability
        prob = prior_probabilities[class_idx]

        # Multiply by conditional probabilities
        for feature_idx, feature_value in enumerate(feature_indices):
            prob *= conditional_probabilities[feature_idx][class_idx, feature_value]

        class_probabilities.append(prob)

        ### Your code here

    # Normalize probabilities
    print(class_probabilities)
    total_prob = sum(class_probabilities) # 0.2 0.3
    if total_prob > 0:
        normalized_probs = [p / total_prob for p in class_probabilities]
    else:
        normalized_probs = [0.5, 0.5]  # Default if all probabilities are 0

    # Make prediction
    predicted_class_idx = np.argmax(class_probabilities)
    prediction = class_names[predicted_class_idx]

    # Create probability dictionary
    prob_dict = {
        'No': round(normalized_probs[0].item(), 2),
        'Yes': round(normalized_probs[1].item(), 2)
    }

    return prediction, prob_dict

In [16]:
X = ['Sunny','Cool', 'High', 'Strong']
prior_probs, conditional_probs, feature_names = train_naive_bayes(train_data)
prediction, prob_dict = predict_tennis(
    X, prior_probs, conditional_probs, feature_names
)
print(f"prediction: {prediction}")

print(f"prob_dict: {prob_dict}")

if prediction == 'Yes':
    print("Ad should go!")
else:
    print("Ad should not go!")

[0.018750000000000003, 0.002777777777777777]
prediction: No
prob_dict: {'No': 0.87, 'Yes': 0.13}
Ad should not go!


## Traffic Data

In [30]:
traffic_data = [['Weekday', 'Spring', 'None', 'None', 'On Time'], 
                ['Weekday', 'Winter', 'None', 'Slight', 'On Time'],
                ['Weekday', 'Winter', 'None', 'None', 'On Time'],
                ['Holiday', 'Winter', 'High', 'Slight', 'Late'],
                ['Saturday', 'Summer', 'Normal', 'None', 'On Time'],
                ['Weekday', 'Autumn', 'Normal', 'None', 'Very Late'],
                ['Holiday', 'Summer', 'High', 'Slight', 'On Time'],
                ['Sunday', 'Summer', 'Normal', 'None', 'On Time'],
                ['Weekday', 'Winter', 'High', 'Heavy', 'Very Late'],
                ['Weekday', 'Summer', 'None', 'Slight', 'On Time'],
                ['Saturday', 'Spring', 'High', 'Heavy', 'Cancelled'],
                ['Weekday', 'Summer', 'High', 'Slight', 'On Time'],
                ['Weekday', 'Winter', 'Normal', 'None', 'Late'],
                ['Weekday', 'Summer', 'High', 'None', 'On Time'],
                ['Weekday', 'Winter', 'Normal', 'Heavy', 'Very Late'],
                ['Saturday', 'Autumn', 'High', 'Slight', 'On Time'],
                ['Weekday', 'Autumn', 'None', 'Heavy', 'On Time'],
                ['Holiday', 'Spring', 'Normal', 'Slight', 'On Time'],
                ['Weekday', 'Spring', 'Normal', 'None', 'On Time'],
                ['Weekday', 'Spring', 'Normal', 'Heavy', 'On Time']]

traffic_data = np.array(traffic_data)

In [31]:
type(train_data)

numpy.ndarray

In [32]:
type(traffic_data)

numpy.ndarray

In [33]:
def compute_prior_probabilities(train_data):
    class_names = ['On Time', 'Late', 'Very Late', 'Cancelled']
    total_samples = len(train_data)
    prior_probs = np.zeros(len(class_names))
    for i, class_name in enumerate(class_names):
        class_count = np.sum(train_data[:, -1] == class_name)
        prior_probs[i] = class_count / total_samples

    return prior_probs

prior_probablity = compute_prior_probabilities(traffic_data)
prior_probablity

array([0.7 , 0.1 , 0.15, 0.05])

In [34]:
def compute_conditional_probabilities(train_data):
    
    class_names = ['On Time', 'Late', 'Very Late', 'Cancelled']

    n_features = train_data.shape[1] - 1  # Exclude target column
    conditional_probs = []
    feature_values = []

    for feature_idx in range(n_features): # 0, 1, 2, 3
        # Get unique values for this feature
        unique_values = np.unique(train_data[:, feature_idx]) # 0: S, O R
        feature_values.append(unique_values)

        # Initialize conditional probability matrix
        feature_cond_probs = np.zeros((len(class_names), len(unique_values)))
        # 0: 2x3

        for class_idx, class_name in enumerate(class_names):
            # Get samples for this class
            class_mask = train_data[:, -1] == class_name
            class_samples = train_data[class_mask]

            # Count occurrences of each feature value in this class

            for value_idx, value in enumerate(unique_values):
                # Count occurrences of this feature value in this class
                # 0: S, O, R
                feature_count = np.sum(class_samples[:, feature_idx] == value)
                # Calculate conditional probability
                feature_cond_probs[class_idx, value_idx] = feature_count / len(class_samples)

        conditional_probs.append(feature_cond_probs)

    return conditional_probs, feature_values

In [35]:
def predict_traffic(X, prior_probabilities, conditional_probabilities, feature_names):

    class_names = ['On Time', 'Late', 'Very Late', 'Cancelled']

    # Get feature indices
    feature_indices = [] # 2, 1, 1, 1
    for i, feature_value in enumerate(X):
        feature_indices.append(get_feature_index(feature_value, feature_names[i]))

    # Calculate probabilities for each class
    class_probabilities = []

    for class_idx in range(len(class_names)):
        # Start with prior probability
        prob = prior_probabilities[class_idx]

        # Multiply by conditional probabilities
        for feature_idx, feature_value in enumerate(feature_indices):
            prob *= conditional_probabilities[feature_idx][class_idx, feature_value]

        class_probabilities.append(prob)

        ### Your code here

    # Normalize probabilities
    print(class_probabilities)
    total_prob = sum(class_probabilities) # 0.2 0.3
    if total_prob > 0:
        normalized_probs = [p / total_prob for p in class_probabilities]
    else:
        normalized_probs = [0.25, 0.25, 0.25, 0.25]  # Default if all probabilities are 0

    # Make prediction
    predicted_class_idx = np.argmax(class_probabilities)
    prediction = class_names[predicted_class_idx]

    # Create probability dictionary
    prob_dict = {
        'On Time': round(normalized_probs[0].item(), 2),
        'Late': round(normalized_probs[1].item(), 2),
        'Very Late': round(normalized_probs[2].item(), 2),
        'Cancelled': round(normalized_probs[3].item(), 2)
    }

    return prediction, prob_dict

In [36]:
X = ['Weekday', 'Winter', 'High', 'Heavy']
prior_probs, conditional_probs, feature_names = train_naive_bayes(traffic_data)
prediction, prob_dict = predict_traffic(
    X, prior_probs, conditional_probs, feature_names
)
print(f"prediction: {prediction}")

print(f"prob_dict: {prob_dict}")

[0.0026239067055393583, 0.0, 0.022222222222222216, 0.0]
prediction: Very Late
prob_dict: {'On Time': 0.11, 'Late': 0.0, 'Very Late': 0.89, 'Cancelled': 0.0}


## Iris Classification


In [51]:
lengths = [1.4, 1.0, 1.3, 1.9, 2.0, 1.8, 3.0, 3.8, 4.1, 3.9, 4.2, 3.4]
classes = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

lengths = np.array(lengths)
classes = np.array(classes)

In [52]:
mean_class_0 = np.mean(lengths[classes == 0])
mean_class_0

1.5666666666666667

In [62]:
variance_class_0 = np.var(lengths[classes == 0], ddof=1)
variance_class_0

0.15466666666666667

In [63]:
mean_class_1 = np.mean(lengths[classes == 1])
mean_class_1

3.733333333333333

In [64]:
variance_class_1 = np.var(lengths[classes == 1], ddof=1)
variance_class_1

0.20666666666666664

In [65]:
import math

def gaussian_probability(x, mean, variance):
    """
    Tính xác suất theo phân phối Gaussian (Normal)
    P(x|μ,σ²) = (1/√(2πσ²)) * exp(-(x-μ)²/(2σ²))
    """
    std_dev = math.sqrt(variance)  # độ lệch chuẩn σ = √variance
    
    # Tính từng thành phần
    coefficient = 1 / (math.sqrt(2 * math.pi) * std_dev)
    exponent = -((x - mean) ** 2) / (2 * variance)
    probability = coefficient * math.exp(exponent)
    
    return probability

In [66]:
x_test = 3.4

prob_class_0 = gaussian_probability(x_test, mean_class_0, variance_class_0)
prob_class_0

1.9378275382105394e-05

In [67]:
prob_class_1 = gaussian_probability(x_test, mean_class_1, variance_class_1)
prob_class_1

0.6707011020792182

In [70]:
total_samples = len(classes)
prior_class_0 = np.sum(classes == 0) / total_samples  # 6/12 = 0.5
prior_class_1 = np.sum(classes == 1) / total_samples  # 6/12 = 0.5

In [73]:
def gaussian_probability(x, mean, variance):
    """Tính P(x|μ,σ²) theo phân phối Gaussian"""
    std_dev = math.sqrt(variance)
    coefficient = 1 / (math.sqrt(2 * math.pi) * std_dev)
    exponent = -((x - mean) ** 2) / (2 * variance)
    return coefficient * math.exp(exponent)

In [82]:
# Điểm cần dự đoán
x_test = 3.4

# Bước 1: Tính likelihood P(X|Class)
likelihood_class_0 = gaussian_probability(x_test, mean_class_0, variance_class_0)
print(f"likelihood_class_0: {likelihood_class_0}")
likelihood_class_1 = gaussian_probability(x_test, mean_class_1, variance_class_1)
print(f"likelihood_class_1: {likelihood_class_1}")

likelihood_class_0: 1.9378275382105394e-05
likelihood_class_1: 0.6707011020792182


In [81]:
# Bước 2: Tính numerator = P(X|Class) × P(Class)
numerator_class_0 = likelihood_class_0 * prior_class_0
print(f"numerator_class_0: {numerator_class_0}")
numerator_class_1 = likelihood_class_1 * prior_class_1
print(f"numerator_class_1: {numerator_class_1}")

numerator_class_0: 9.689137691052697e-06
numerator_class_1: 0.3353505510396091


In [84]:
# Bước 3: Tính marginal probability P(X)
marginal_prob = numerator_class_0 + numerator_class_1
print(f"marginal_prob: {marginal_prob}")

marginal_prob: 0.33536024017730015


In [85]:
# Bước 4: Tính posterior P(Class|X)
posterior_class_0 = numerator_class_0 / marginal_prob
print(f"posterior_class_0: {posterior_class_0}")
posterior_class_1 = numerator_class_1 / marginal_prob
print(f"posterior_class_1: {posterior_class_1}")

posterior_class_0: 2.889173053409727e-05
posterior_class_1: 0.9999711082694659


In [87]:
print(f"P(Class=0|X) ∝ {numerator_class_0:.2e}")
print(f"P(Class=1|X) ∝ {numerator_class_1:.4f}")

P(Class=0|X) ∝ 9.69e-06
P(Class=1|X) ∝ 0.3354


## END